In [5]:
#useful py libraries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as ticker

# Time series tools

In [13]:
def plot_climat_monthly(ds):
    """
    starting with an xarray dataset with the fields ave and st_dev seperated by month, 
    produce a plot with a 95% confidence interval and mean
    
    
    
    example use:
        plt.figure(figsize=[8,8])
        plt.subplot()
        plot_climat(xarray_dataset)
        plt.show()
    
    """
    
    #start in december, go through one whole year and then end in jan
    
    month_nums=np.array([ds.month[0].values-0.5,*(ds.month+0.5).values,ds.month[-1].values+1.5])
    aves=np.array([ds.ave[-1].values,*ds.ave.values,ds.ave[0].values])
    st_devs=np.array([ds.st_dev[-1].values,*ds.st_dev.values,ds.st_dev[0].values])
    
    plt.plot(
        month_nums,
        aves, 
        color='black',
        label='Mean'
    )

    plt.fill_between(
        month_nums,
        aves-2*st_devs, 
        aves+2*st_devs,
        color='lightgrey',
        label='95% CI'
    )

    plt.xticks(
        ticks=[1,2,3,4,5,6,7,8,9,10,11,12],
        labels=months
    )
    
    plt.xlim(1,13)

    plt.ylabel('km\u00b2')
    plt.legend()
 

In [14]:
def plot_year(da, year):
    """
    starting with an xarray dataarray timeseries, and a specified year, 
    plot the data as a monthly timeseries
    
    example use:
        plt.figure(figsize=[8,8])
        plt.subplot()
        plot_year(xarray_dataArray)
        plt.show()
    
    """
    
    #TO_DO: extent with data from previous December and following Jan?
    
    to_plot_da=da.where(da.time.dt.year==int(year), drop=True)
    
    #if monthly, then plot ticks are middle of the month
    if len(to_plot_da.time)==to_plot_da.time.dt.month.values[-1]: #number of months equals the month of the last timestamp
        time_ticks=to_plot_da.time.dt.month+0.5
    else:
        #daily, plot ticks are month + fraction of month for that day
        time_ticks=to_plot_da.time.dt.month+(to_plot_da.time.dt.day/to_plot_da.time.dt.daysinmonth)
    
    plt.plot(
        time_ticks,
        to_plot_da,
        color='purple',
        dashes=(4,3),
        label=str(int(year))
    )
    
    plt.xticks(
        ticks=[1,2,3,4,5,6,7,8,9,10,11,12],
        labels=months
    )
    
    plt.xlim(1,13)
    
    plt.legend()

# Geo-spatial tools

In [1]:
def plot_stipling(da,criteria,ax):
    """
    
    
    
    """
    
    #try:
        #if len(da.model.values)<2:
        #    raise EnvironmentError("Nothing to stiple, there needs to be at least 2 models")
    #except:
        #raise EnvironmentError("Does this dataarray have a model dimension?")
    #Error checking should probably check there are only three dimensions.
    
    lessThan=(da<-criteria)
    greatThan=(da>criteria)
    
    #Area to add stipling 
    combined=(
        lessThan|greatThan
    )

    #plot it
    plt.sca(ax)
    
    c=plt.pcolor(
        combined.x,
        combined.y,
        np.ma.masked_equal(combined,0),
        hatch='.',
        alpha=0,
        shading='auto',
        transform=ccrs.SouthPolarStereo(true_scale_latitude=-70),
    )
    
    return combined
